In [1]:
import os, sys
import numpy as np
import pandas as pd
sys.path.insert(0,r"C:\Users\rniswon\Documents\Data\Git\flopy.git")
import gsflow


#---------------------------------------------------------
# Read modflow only
#---------------------------------------------------------

control_file = r"C:\Users\rniswon\Documents\Data\Git\carmel.git\Simulation_20160812T131753\carmel_test.control"

gs = gsflow.GsflowModel.load_from_file(control_file = control_file, mf_load_only = ['DIS', 'BAS6', 'SFR'],
                                       modflow_only=True )
sfr = gs.mf.sfr

flopy is installed in C:\Users\rniswon\Documents\Data\Git\flopy.git\flopy
Control file is loaded
Working on loading MODFLOW files ....
MODFLOW files are loaded ... 


In [2]:
gs.mf.get_package_list()

['DIS', 'BAS6', 'SFR']

In [3]:
import flopy
flopy.__file__

'C:\\Users\\rniswon\\Documents\\Data\\Git\\flopy.git\\flopy\\__init__.pyc'

In [4]:
sys.executable

'C:\\Users\\rniswon\\AppData\\Local\\Continuum\\miniconda2\\python.exe'

In [14]:

# ------------------------------------------------------------------
# Read a csv file with data that has two columns. The first is the iseg and the
# second is the elevation change.
#-------------------------------------------------------------------
file_with_elev_changes = r"C:\Users\rniswon\Documents\Data\Git\carmel.git\notebook\change_sfr_elev.csv"
elv_chg_df = pd.read_csv(file_with_elev_changes)
reach_data = sfr.reach_data.copy() # get a copy of sfr object

for i, seg in elv_chg_df.iterrows():
    current_seg = seg['ISEG']
    current_change = seg['DELTA']
    filter_by_seg = reach_data['iseg'] == current_seg
    reach_data['strtop'][filter_by_seg] = reach_data['strtop'][filter_by_seg] + current_change

gs.mf.sfr.reach_data = reach_data

gs.mf.change_model_ws(r"C:\Users\rniswon\Documents\Data\Git\carmel.git\notebook\myyy")
gs.mf.sfr.write_file()
reach_data = gs.mf.sfr.reach_data


changing model workspace...
   C:\Users\rniswon\Documents\Data\Git\carmel.git\notebook\myyy


In [24]:
dis = gs.mf.dis
top = dis.gettop()
for i, seg in elv_chg_df.iterrows():
    current_change = seg['DELTA']
    current_jrch = int(seg['JRCH'])
    current_irch = int(seg['IRCH'])
    top[current_irch,current_jrch] = top[current_irch,current_jrch] + current_change
    
gs.mf.dis.write_file()
top = gs.mf.dis.top